In [133]:
import pandas as pd
import numpy as np
file_path="data.xlsx"
xls=pd.ExcelFile(file_path)

In [134]:
df_New_Account=xls.parse("New_Account")
df_New_Card=xls.parse("New_Card")
df_District=xls.parse("District")
df_Loan=xls.parse("Loan")
df_New_Client=xls.parse("New_Client")
df_New_Disposition=xls.parse("New_Disposition")
df_New_Transaction=xls.parse("New_Transaction")
df_Order=xls.parse("Order")

In [135]:
#Data-CLeaning-1
df_New_Transaction.drop_duplicates(inplace=True)
df_Order.drop_duplicates(inplace=True)

In [136]:
#FILLING MISSING VALUES #Data-Cleaning-2
# Fill 'operation' where k_symbol is 'OLD AGE PENSION' or 'INTEREST CREDITED' and operation is null
df_New_Transaction.loc[
    (df_New_Transaction['k_symbol'].isin(['OLD AGE PENSION', 'INTEREST CREDITED'])) &
    (df_New_Transaction['operation'].isna()),
    'operation'
] = 'COLLECTION FROM ANOTHER BANK'


In [137]:
#DATA CLEANING -3
# Step 1: Get rows in df_Order where k_symbol is missing
# Step 2: Merge with df_New_Transaction on account_id and amount
# Step 3: Fill missing k_symbol in df_Order where match found
# Step 4: Fill remaining missing values with 'UNKNOWN'
order_missing_k = df_Order[df_Order['k_symbol'].isna()].copy()

merged_order = order_missing_k.merge(
    df_New_Transaction[['account_id', 'amount', 'k_symbol']],
    on=['account_id', 'amount'],
    how='left',
    suffixes=('', '_txn')
)
for idx, row in merged_order.iterrows():
    if pd.notna(row['k_symbol_txn']):
        df_Order.loc[
            (df_Order['account_id'] == row['account_id']) &
            (df_Order['amount'] == row['amount']) &
            (df_Order['k_symbol'].isna()),
            'k_symbol'
        ] = row['k_symbol_txn']

df_Order['k_symbol'] = df_Order['k_symbol'].fillna('UNKNOWN')


In [139]:
#DATA CLEANING-4 giving meaningful treatment to the k_symbol in New_Transaction dataframe

# Step 1: Get rows in df_New_Transaction where k_symbol is missing
txn_missing_k = df_New_Transaction[df_New_Transaction['k_symbol'].isna()].copy()

# Step 2: Merge with df_Order on account_id and amount
merged_txn = txn_missing_k.merge(
    df_Order[['account_id', 'amount', 'k_symbol']],
    on=['account_id', 'amount'],
    how='left',
    suffixes=('', '_order')
)

# Step 3: Fill missing k_symbol in df_New_Transaction where match found
for idx, row in merged_txn.iterrows():
    if pd.notna(row['k_symbol_order']):
        df_New_Transaction.loc[
            (df_New_Transaction['account_id'] == row['account_id']) &
            (df_New_Transaction['amount'] == row['amount']) &
            (df_New_Transaction['k_symbol'].isna()),
            'k_symbol'
        ] = row['k_symbol_order']

# Step 4: Fill remaining missing values with 'UNKNOWN'
df_New_Transaction['k_symbol'] = df_New_Transaction['k_symbol'].fillna('UNKNOWN')


In [140]:
def inspect_df(df, name):
    print(f"\n {name} ------------------")
    print("Missing values:\n", df.isnull().sum())
    print("\nData types:\n", df.dtypes)

# Inspect each DataFrame
inspect_df(df_New_Account, "New_Account")
inspect_df(df_New_Transaction, "New_Transaction")
inspect_df(df_New_Client, "New_Client")
inspect_df(df_New_Disposition, "New_Disposition")
inspect_df(df_New_Card, "New_Card")
inspect_df(df_District, "District")
inspect_df(df_Loan, "Loan")
inspect_df(df_Order, "Order")


 New_Account ------------------
Missing values:
 account_id     0
district_id    0
frequency      0
dtype: int64

Data types:
 account_id      int64
district_id     int64
frequency      object
dtype: object

 New_Transaction ------------------
Missing values:
 Unnamed: 0    0
account_id    0
date          0
type          0
operation     0
amount        0
balance       0
k_symbol      0
dtype: int64

Data types:
 Unnamed: 0      int64
account_id      int64
date            int64
type           object
operation      object
amount        float64
balance       float64
k_symbol       object
dtype: object

 New_Client ------------------
Missing values:
 Unnamed: 0      0
client_id       0
birth_number    0
district_id     0
gender          0
age             0
age_levels      0
dtype: int64

Data types:
 Unnamed: 0       int64
client_id        int64
birth_number     int64
district_id      int64
gender          object
age              int64
age_levels      object
dtype: object

 New_Dispositio

In [ ]:
pip install pandas plotly dash

In [5]:
#preparing dashboard account wise 
df_District["A3"].unique()

array(['Prague', 'central Bohemia', 'south Bohemia', 'west Bohemia',
       'north Bohemia', 'east Bohemia', 'south Moravia', 'north Moravia'],
      dtype=object)

In [53]:
df_District.rename(columns={"A1":"district_id","A3": "district_name"},inplace=True)
merged_df=pd.merge(df_New_Account,df_District,on="district_id")
prague_accounts=merged_df[merged_df["district_name"]=="Prague"]
acc_id=prague_accounts["account_id"]
credit_prague_acc=df_New_Transaction[(df_New_Transaction["account_id"].isin(acc_id))&(df_New_Transaction["type"]=="CREDIT")].groupby(df_New_Transaction["account_id"]).size().reset_index(name="credit_transaction_count")
print(credit_prague_acc)
#for information
credit_prague_info=df_New_Transaction[(df_New_Transaction["account_id"].isin(acc_id))&(df_New_Transaction["type"]=="CREDIT")]
credit_prague_info=credit_prague_info.sort_values(by="account_id")
print(credit_prague_info)
credit_prague_acc=credit_prague_acc.sort_values(by="credit_transaction_count",ascending=False)


     account_id  credit_transaction_count
0             2                       151
1            17                        32
2            22                       117
3            36                       133
4            49                        42
..          ...                       ...
549       11021                       232
550       11111                       185
551       11141                        66
552       11231                        47
553       11349                        90

[554 rows x 2 columns]
         Unnamed: 0  account_id    date    type                     operation  \
4132           4133           2  930612  CREDIT  COLLECTION FROM ANOTHER BANK   
85582         85583           2  940831  CREDIT                           NaN   
193955       193956           2  950731  CREDIT                           NaN   
876807       876808           2  980531  CREDIT                           NaN   
7707           7708           2  930731  CREDIT                    

In [256]:
#Task-1(Account-wise)
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, Layout
from IPython.display import display

df_District.rename(columns={"A1": "district_id", "A3": "district_name"}, inplace=True)
merged_df = pd.merge(df_New_Account, df_District, on="district_id")
df_New_Transaction["type"] = df_New_Transaction["type"].str.strip().str.upper()
merged_df["district_name"] = merged_df["district_name"].str.strip()

def get_credit_data(district):
    district_accounts = merged_df[merged_df["district_name"].str.lower() == district.lower()]
    account_ids = district_accounts["account_id"]
    credit_data = df_New_Transaction[
        (df_New_Transaction["account_id"].isin(account_ids)) &
        (df_New_Transaction["type"] == "CREDIT")
    ]
    grouped = credit_data.groupby("account_id").size().reset_index(name="credit_transaction_count")
    grouped = grouped.sort_values(by="credit_transaction_count", ascending=False)
    return grouped, credit_data

def dashboard(district_name):
    acc_df, full_info = get_credit_data(district_name)
    account_selector = widgets.SelectMultiple(
        options=acc_df["account_id"].astype(str),
        value=tuple(acc_df["account_id"].astype(str).tolist()[:10]),
        description="Accounts",
        layout=Layout(width="50%")
    )
    def plot_selected_accounts(selected_ids):
        filtered = acc_df[acc_df["account_id"].astype(str).isin(selected_ids)]
        if filtered.empty:
            print("No data for selected accounts.")
            return
        plt.figure(figsize=(10, 5))
        plt.bar(filtered["account_id"].astype(str), filtered["credit_transaction_count"], color="royalblue")
        plt.xlabel("Account ID")
        plt.ylabel("Credit Transaction Count")
        plt.title(f"Credit Transactions – {district_name}")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    interact(plot_selected_accounts, selected_ids=account_selector)

district_dropdown = widgets.Dropdown(
    options=['Prague','south Moravia','north Moravia'],
    description="District",
    layout=Layout(width="50%")
)

interact(dashboard, district_name=district_dropdown)


AttributeError: 'DataFrame' object has no attribute 'str'

In [80]:
#Task-1(month-wise)
import pandas as pd

df_New_Transaction = pd.read_excel("data.xlsx", sheet_name="New_Transaction", dtype=str)
df_New_Transaction.columns = df_New_Transaction.columns.str.strip()
df_New_Transaction["date"] = pd.to_datetime(df_New_Transaction["date"], format="%y%m%d", errors="coerce")
df_New_Transaction["year"] = df_New_Transaction["date"].dt.year
df_New_Transaction["month"] = df_New_Transaction["date"].dt.month
distinct_years = df_New_Transaction["year"].dropna().unique()
distinct_months = df_New_Transaction["month"].dropna().unique()
distinct_years.sort()
distinct_months.sort()
print("Distinct years:", distinct_years)
print("Distinct months:", distinct_months)
#to know the credit count transaction by month for district wise

df_District.rename(columns={"A1": "district_id", "A3": "district_name"}, inplace=True)
merged_df = pd.merge(df_New_Account, df_District, on="district_id")

df_New_Transaction["account_id"] = df_New_Transaction["account_id"].astype(int)
merged_df["account_id"] = merged_df["account_id"].astype(int)

full_df = pd.merge(df_New_Transaction, merged_df, on="account_id")
selected_districts=['Prague','south Moravia','north Moravia']
credit_by_district = (
    full_df[(full_df["type"] == "CREDIT")&(full_df["district_name"].isin(selected_districts)) ]
    .groupby("district_name")
    .size()
    .reset_index(name="credit_transaction_count")
    .sort_values(by="credit_transaction_count", ascending=False)
)

print(credit_by_district)


Distinct years: [1993 1994 1995 1996 1997 1998]
Distinct months: [ 1  2  3  4  5  6  7  8  9 10 11 12]


In [258]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load and prepare data
df_New_Transaction = pd.read_excel("data.xlsx", sheet_name="New_Transaction", dtype=str)
df_New_Account = pd.read_excel("data.xlsx", sheet_name="New_Account")
df_District = pd.read_excel("data.xlsx", sheet_name="District")

df_New_Transaction.columns = df_New_Transaction.columns.str.strip()
df_New_Transaction["date"] = pd.to_datetime(df_New_Transaction["date"], format="%y%m%d", errors="coerce")
df_New_Transaction["year"] = df_New_Transaction["date"].dt.year
df_New_Transaction["month"] = df_New_Transaction["date"].dt.month

df_District.rename(columns={"A1": "district_id", "A3": "district_name"}, inplace=True)
merged_df = pd.merge(df_New_Account, df_District, on="district_id")

df_New_Transaction["account_id"] = df_New_Transaction["account_id"].astype(int)
merged_df["account_id"] = merged_df["account_id"].astype(int)

full_df = pd.merge(df_New_Transaction, merged_df, on="account_id")

selected_districts = ["Prague", "south Moravia", "north Moravia"]

# Dropdowns for filtering
year_options = sorted(full_df["year"].dropna().unique())
month_options = sorted(full_df["month"].dropna().unique())

year_dropdown = widgets.Dropdown(options=year_options, description="Year:")
month_dropdown = widgets.Dropdown(options=month_options, description="Month:")

output = widgets.Output()

def update_dashboard(change):
    with output:
        clear_output()
        year = year_dropdown.value
        month = month_dropdown.value

        filtered_df = full_df[
            (full_df["type"] == "CREDIT") &
            (full_df["district_name"].isin(selected_districts)) &
            (full_df["year"] == year) &
            (full_df["month"] == month)
        ]

        credit_by_district = (
            filtered_df.groupby("district_name")
            .size()
            .reset_index(name="credit_transaction_count")
            .sort_values(by="credit_transaction_count", ascending=False)
        )

        display(f"Credit transactions for {month}/{year}")
        display(credit_by_district)

year_dropdown.observe(update_dashboard, names="value")
month_dropdown.observe(update_dashboard, names="value")

display(widgets.HBox([year_dropdown, month_dropdown]))
display(output)

# Trigger display once at start
update_dashboard(None)


Output()

In [257]:
#task-1(acc+month)
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Load and clean data
df_New_Transaction = pd.read_excel("data.xlsx", sheet_name="New_Transaction", dtype=str)
df_New_Account = pd.read_excel("data.xlsx", sheet_name="New_Account")
df_District = pd.read_excel("data.xlsx", sheet_name="District")

df_New_Transaction.columns = df_New_Transaction.columns.str.strip()
df_New_Account.columns = df_New_Account.columns.str.strip()
df_District.columns = df_District.columns.str.strip()

df_New_Transaction["date"] = pd.to_datetime(df_New_Transaction["date"], format="%y%m%d", errors="coerce")
df_New_Transaction["year"] = df_New_Transaction["date"].dt.year
df_New_Transaction["month"] = df_New_Transaction["date"].dt.month

df_New_Transaction["account_id"] = df_New_Transaction["account_id"].astype(int)
df_New_Account["account_id"] = df_New_Account["account_id"].astype(int)
df_New_Account["district_id"] = df_New_Account["district_id"].astype(int)
df_District["A1"] = df_District["A1"].astype(int)
df_District.rename(columns={"A1": "district_id", "A3": "district_name"}, inplace=True)

merged_accounts = pd.merge(df_New_Account, df_District, on="district_id")
full_df = pd.merge(df_New_Transaction, merged_accounts, on="account_id")
full_df["district_name"] = full_df["district_name"].str.strip()
full_df["type"] = full_df["type"].str.strip()

# Widgets
years = sorted(full_df["year"].dropna().unique())
months = sorted(full_df["month"].dropna().unique())
districts = sorted(full_df["district_name"].dropna().unique())
selected_districts=['Prague','south Moravia','north Moravia']
year_dropdown = widgets.Dropdown(options=years, description="Year:")
month_dropdown = widgets.Dropdown(options=months, description="Month:")
district_dropdown = widgets.Dropdown(options=selected_districts, description="District:")
account_multiselect = widgets.SelectMultiple(options=[], description="Accounts:", rows=10)

output = widgets.Output()

# Dynamic account filtering
def update_account_options(*args):
    y = year_dropdown.value
    m = month_dropdown.value
    d = district_dropdown.value

    filtered_accounts = full_df[
        (full_df["year"] == y) &
        (full_df["month"] == m) &
        (full_df["district_name"] == d)
    ]["account_id"].unique()

    account_multiselect.options = sorted(filtered_accounts.tolist())

def update_plot(change=None):
    with output:
        clear_output()
        
        y = year_dropdown.value
        m = month_dropdown.value
        d = district_dropdown.value
        accs = list(account_multiselect.value)

        data = full_df[
            (full_df["year"] == y) &
            (full_df["month"] == m) &
            (full_df["district_name"] == d) &
            (full_df["account_id"].isin(accs)) &
            (full_df["type"] == "CREDIT")
        ]

        result = (
            data.groupby("account_id")
            .size()
            .reset_index(name="credit_transaction_count")
            .sort_values(by="credit_transaction_count", ascending=False)
        )

        if result.empty:
            print("No data for selected filters.")
        else:
            plt.figure(figsize=(10, 5))
            plt.bar(result["account_id"].astype(str), result["credit_transaction_count"], color="skyblue")
            plt.xlabel("Account ID")
            plt.ylabel("Credit Transaction Count")
            plt.title(f"CREDIT Transactions - {d}, {m}/{y}")
            plt.xticks(rotation=45)
            plt.grid(axis="y")
            plt.tight_layout()
            plt.show()

# Link filters
year_dropdown.observe(update_account_options, names="value")
month_dropdown.observe(update_account_options, names="value")
district_dropdown.observe(update_account_options, names="value")
account_multiselect.observe(update_plot, names="value")

# Display dashboard
display(widgets.HBox([year_dropdown, month_dropdown, district_dropdown]))
display(account_multiselect)
display(output)

# Initialize options and plot
update_account_options()
update_plot()


SelectMultiple(description='Accounts:', options=(), rows=10, value=())

Output()

In [99]:
df_District["A2"].nunique()

77

In [22]:
df_District.rename(columns={"A2":"district_name","A5":"lt_499","A6":"btw_500_1999","A7": "btw_2000_9999","A8":"gt_10000"},inplace=True)
avg_pop = {
    'lt_499': 250,
    'btw_500_1999': 1250,
    'btw_2000_9999': 6000,
    'gt_10000': 15000
}

df_District['Estimated_Population'] = (
    df_District['lt_499'] * avg_pop['lt_499'] +
    df_District['btw_500_1999'] * avg_pop['btw_500_1999'] +
    df_District['btw_2000_9999'] * avg_pop['btw_2000_9999'] +
    df_District['gt_10000'] * avg_pop['gt_10000']
)

most_populated = df_District.sort_values(by='Estimated_Population', ascending=False).head(5)
least_populated = df_District.sort_values(by='Estimated_Population', ascending=True).head(5)

print("Top 5 Most Populated Districts:")
print(most_populated[['district_name', 'Estimated_Population']])

print("\nTop 5 Least Populated Districts:")
print(least_populated[['district_name', 'Estimated_Population']])
df_New_Transaction['date'] =df_New_Transaction['date'].astype(int)

import pandas as pd
df_New_Transaction['date_str'] = df_New_Transaction['date'].astype(str).str.zfill(6)
df_New_Transaction['date_parsed'] = pd.to_datetime('19' + df_New_Transaction['date_str'], format='%Y%m%d', errors='coerce')
filtered_date = df_New_Transaction[
    (df_New_Transaction['date_parsed'].dt.year == 1998) &
    (df_New_Transaction['date_parsed'].dt.month.isin([10, 11, 12]))
]
print(filtered_date)

Top 5 Most Populated Districts:
      district_name  Estimated_Population
56          Hodonin                216500
54    Brno - venkov                207750
67  Frydek - Mistek                191750
71          Olomouc                172500
70       Novy Jicin                171000

Top 5 Least Populated Districts:
      district_name  Estimated_Population
0       Hl.m. Praha                 15000
73  Ostrava - mesto                 15000
53     Brno - mesto                 15000
25    Plzen - mesto                 15000
30           Tachov                 56500
         Unnamed: 0  account_id    date   type        operation   amount  \
981737       981738        2506  981001  DEBIT  CASH WITHDRAWAL   8400.0   
981738       981739        2726  981001  DEBIT  CASH WITHDRAWAL  18400.0   
981739       981740        3560  981001  DEBIT  CASH WITHDRAWAL   5100.0   
981740       981741        2236  981001  DEBIT  CASH WITHDRAWAL  12299.0   
981741       981742        2367  981001  DEBIT  CA

In [142]:
#task-2
# STEP 1: Rename & estimate population (you already did this)
df_District.rename(columns={"A1":"district_id","A2": "district_name", "A5": "lt_499", "A6": "btw_500_1999", 
                            "A7": "btw_2000_9999", "A8": "gt_10000"}, inplace=True)

avg_pop = {
    'lt_499': 250,
    'btw_500_1999': 1250,
    'btw_2000_9999': 6000,
    'gt_10000': 15000
}

df_District['Estimated_Population'] = (
    df_District['lt_499'] * avg_pop['lt_499'] +
    df_District['btw_500_1999'] * avg_pop['btw_500_1999'] +
    df_District['btw_2000_9999'] * avg_pop['btw_2000_9999'] +
    df_District['gt_10000'] * avg_pop['gt_10000']
)

# STEP 2: Get top & bottom districts
most_populated = df_District.sort_values(by='Estimated_Population', ascending=False).head(5)
least_populated = df_District.sort_values(by='Estimated_Population').head(5)

top_districts = most_populated['district_id'].tolist()
bottom_districts = least_populated['district_id'].tolist()

# STEP 3: Parse and filter dates
df_New_Transaction['date'] = df_New_Transaction['date'].astype(str).str.zfill(6)
df_New_Transaction['date_parsed'] = pd.to_datetime('19' + df_New_Transaction['date'], format='%Y%m%d', errors='coerce')

filtered_txn = df_New_Transaction[
    (df_New_Transaction['date_parsed'].dt.year == 1998) &
    (df_New_Transaction['date_parsed'].dt.month.isin([10, 11, 12]))
]

# STEP 4: Merge transactions with accounts and districts
merged = (
    filtered_txn
    .merge(df_New_Account, on='account_id', how='left')
    .merge(df_District[['district_id', 'district_name', 'Estimated_Population']], on='district_id', how='left')
)

# STEP 5: Filter to top/bottom districts & summarize
top_txns = merged[merged['district_id'].isin(top_districts)]
bottom_txns = merged[merged['district_id'].isin(bottom_districts)]

# STEP 6: Group and sum
top_summary = top_txns.groupby('type')['amount'].sum()
bottom_summary = bottom_txns.groupby('type')['amount'].sum()

# STEP 7: Display
print("📊 Top 5 Most Populated Districts (Oct-Dec 1998):")
print(top_summary)

print("\n📉 Bottom 5 Least Populated Districts (Oct-Dec 1998):")
print(bottom_summary)


📊 Top 5 Most Populated Districts (Oct-Dec 1998):
type
CREDIT    16607836.2
DEBIT     14536180.2
Name: amount, dtype: float64

📉 Bottom 5 Least Populated Districts (Oct-Dec 1998):
type
CREDIT    47612835.5
DEBIT     42486323.5
Name: amount, dtype: float64


In [38]:
#task-3
a=df_New_Card[df_New_Card["card_id"].duplicated()]
print(a)
merge_disp=pd.merge(df_New_Card,df_New_Disposition,on="disp_id")
full_merge=pd.merge(merge_disp,df_New_Client,on="client_id")
full_merge
crd_mid_age_female=full_merge[(full_merge["card_id"] !=0)&(full_merge["gender"]=="FEMALE")&(full_merge["age_levels"]=="MIDDLE AGED")].groupby("age_levels").size().reset_index(name="count")
print(crd_mid_age_female)

    age_levels  count
0  MIDDLE AGED    227


In [64]:
#task-4
df_District.rename(columns={"A11":"avg_salary"},inplace=True)
district=df_District[(df_District["avg_salary"])>9000]
#print(district[["district_name","avg_salary","district_id"]])

merge_one=pd.merge(df_District,df_New_Account,on="district_id")
merge_two=pd.merge(merge_one,df_New_Disposition,on="account_id")
merge_final=pd.merge(merge_two,df_New_Card,on="disp_id")
#print(merge_final["district_name"])

data=merge_final[(merge_final["district_name"].isin(district["district_name"]))].groupby("district_name").size().reset_index(name="count").sort_values(by="count",ascending=False)
print(data)

          district_name  count
6           Hl.m. Praha    132
8               Karvina     24
16      Ostrava - mesto     22
26                 Zlin     17
17            Pardubice     17
11              Liberec     17
12           Litomerice     16
0          Brno - mesto     16
10                Kolin     15
5       Frydek - Mistek     15
25       Usti nad Labem     12
14       Mlada Boleslav     10
3         Cesky Krumlov     10
9                Kladno     10
18        Plzen - mesto     10
2      Ceske Budejovice      9
23                Tabor      9
4              Chomutov      9
24              Teplice      9
21  Rychnov nad Kneznou      9
15                 Most      8
20        Praha - zapad      8
22              Sokolov      8
13               Melnik      7
19       Praha - vychod      7
1            Ceska Lipa      6
7        Hradec Kralove      5


In [121]:
#task-5
df_District.rename(columns={"A15":"code95"},inplace=True)
df_District["code95"]=df_District["code95"].replace("?",0).astype(int)
district_names=df_District[(df_District["code95"]>6000)]
merged_o=pd.merge(df_District,df_New_Account,on="district_id")
merge_full=pd.merge(merged_o,df_Loan,on="account_id")
data=merge_full[(merge_full["district_name"].isin(district_names["district_name"]))].groupby("district_name").size().reset_index(name="count")
print(data.head(10))

      district_name  count
0      Brno - mesto     24
1  Ceske Budejovice      8
2       Hl.m. Praha     84
3           Karvina     24
4           Olomouc     14
5   Ostrava - mesto     20
6         Pardubice     10
7     Plzen - mesto      6
8           Teplice      6
9    Usti nad Labem      3


In [128]:
#task-6
df_District.rename(columns={"A12":"urcode95","A13":"urcode96"},inplace=True)
unemployment_cols = ["urcode95","urcode96"]
df_District[unemployment_cols] = df_District[unemployment_cols].replace("?", 0).astype(float)
high_unemp_districts = df_District[df_District[unemployment_cols].gt(2).any(axis=1)]
high_unemp_accounts=pd.merge(high_unemp_districts,df_New_Account,on="district_id")
merged_col=pd.merge(high_unemp_accounts,df_New_Transaction,on="account_id")
data=merged_col[(merged_col["operation"])=="COLLECTION FROM ANOTHER BANK"]
data["amount"].sum()

571773129.0

In [146]:
#task-7
import pandas as pd

# Step 1: Clean and prepare data
df_District.rename(columns={"A2": "district_name"}, inplace=True)

# Step 2: Merge transaction → account → district
txn_merge1 = pd.merge(df_New_Transaction, df_New_Account, on='account_id')
txn_merge2 = pd.merge(txn_merge1, df_District[['district_id', 'district_name']], on='district_id')

# Step 3: Merge with disposition → client
txn_disp = pd.merge(txn_merge2, df_New_Disposition[['account_id', 'client_id']], on='account_id')
full_txn = pd.merge(txn_disp, df_New_Client, on='client_id')

# Step 4: Analyze top-paying districts and custome profiles for each operation
operations = ['OLD AGE PENSION', 'HOUSEHOLD', 'INSURANCE PAYMENT', 'LOAN PAYMENT']
results = {}

for op in operations:
    op_data = full_txn[full_txn['k_symbol'] == op]
    if op_data.empty:
        continue

    # Find district with max total amount
    district_sum = op_data.groupby('district_name')['amount'].sum().reset_index()
    max_row = district_sum.sort_values(by='amount', ascending=False).iloc[0]
    max_district = max_row['district_name']
    max_amount = max_row['amount']

    # Filter customer profile from top district
    customer_data = op_data[op_data['district_name'] == max_district]
    customer_profile = customer_data[['client_id', 'birth_number', 'district_name', 'amount']]

    results[op] = {
        'district': max_district,
        'total_amount': max_amount,
        'customers': customer_profile
    }

# Step 5: Display result
for op, info in results.items():
    print(f"\n Operation: {op}")
    print(f"  Max Paying District: {info['district']}")
    print(f" Total Amount Paid: {info['total_amount']}")
    print(" Customers Profile:")
    print(info['customers'].tail(5))



📌 Operation: OLD AGE PENSION
🏙️  Max Paying District: Hl.m. Praha
💰 Total Amount Paid: 27306272.0
👥 Customers Profile:
        client_id  birth_number district_name  amount
254618        767        230616   Hl.m. Praha  4373.0
254622        767        230616   Hl.m. Praha  4373.0
254627        767        230616   Hl.m. Praha  4373.0
254632        767        230616   Hl.m. Praha  4373.0
254637        767        230616   Hl.m. Praha  4373.0

📌 Operation: HOUSEHOLD
🏙️  Max Paying District: Hl.m. Praha
💰 Total Amount Paid: 77802766.0
👥 Customers Profile:
        client_id  birth_number district_name  amount
255351       3966        691007   Hl.m. Praha  1722.0
255357       3966        691007   Hl.m. Praha  1722.0
255364       3966        691007   Hl.m. Praha  1722.0
255371       3966        691007   Hl.m. Praha  1722.0
255378       3966        691007   Hl.m. Praha  1722.0

📌 Operation: INSURANCE PAYMENT
🏙️  Max Paying District: Hl.m. Praha
💰 Total Amount Paid: 6048829.0
👥 Customers Profil

In [149]:
#task-8
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Step 1: Rename for clarity (if not already done)
df_District.rename(columns={"A2": "district_name"}, inplace=True)

# Step 2: Merge data to get client details along with loan and district
merge1 = pd.merge(df_New_Account, df_District[['district_id', 'district_name']], on='district_id')
merge2 = pd.merge(merge1, df_Loan, on='account_id')
merge3 = pd.merge(merge2, df_New_Disposition[['account_id', 'client_id']], on='account_id')
full_data = pd.merge(merge3, df_New_Client[['client_id', 'birth_number']], on='client_id')

# Step 3: Dropdown widgets
district_options = sorted(full_data['district_name'].unique())
status_options = sorted(full_data['status'].unique())

district_dropdown = widgets.Dropdown(
    options=district_options,
    description='District:',
    style={'description_width': 'initial'}
)

status_dropdown = widgets.Dropdown(
    options=status_options,
    description='Loan Status:',
    style={'description_width': 'initial'}
)

# Step 4: Filtering function
def filter_data(district, status):
    filtered = full_data[
        (full_data['district_name'] == district) &
        (full_data['status'] == status)
    ]
    display(filtered[['client_id', 'birth_number', 'status', 'district_name']])

# Step 5: Interactive output
ui = widgets.VBox([district_dropdown, status_dropdown])
out = widgets.interactive_output(
    filter_data, 
    {'district': district_dropdown, 'status': status_dropdown}
)

display(ui, out)


Output()

In [152]:
#task-9
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Step 1: Rename for clarity if not done already
df_District.rename(columns={"A2": "district_name", "A11": "avg_salary", "A15": "code95", "A12": "urcode95", "A13": "urcode96"}, inplace=True)

# Step 2: Clean data (handle missing or string values)
df_District['code95'] = df_District['code95'].replace("?", 0).astype(int)
df_District[['urcode95', 'urcode96']] = df_District[['urcode95', 'urcode96']].replace("?", 0).astype(float)
df_District['avg_salary'] = df_District['avg_salary'].replace("?", 0).astype(float)

# Step 3: Merge datasets (Account → District → Loan → Disposition → Client)
merge1 = pd.merge(df_New_Account, df_District, on='district_id')
merge2 = pd.merge(merge1, df_Loan, on='account_id')
merge3 = pd.merge(merge2, df_New_Disposition[['account_id', 'client_id']], on='account_id')
full_data = pd.merge(merge3, df_New_Client[['client_id', 'birth_number']], on='client_id')

# Step 4: Dropdowns
district_options = sorted(full_data['district_name'].unique())
status_options = sorted(full_data['status'].unique())

district_dropdown = widgets.Dropdown(options=district_options, description='District:')
status_dropdown = widgets.Dropdown(options=status_options, description='Loan Status:')

# Step 5: Filtering + Enriched Display
def show_filtered_info(district, status):
    filtered = full_data[
        (full_data['district_name'] == district) & 
        (full_data['status'] == status)
    ]
    
    if filtered.empty:
        print("No clients found for the selected filters.")
        return
    
    # Display district socio-economic info
    district_info = df_District[df_District['district_name'] == district][['district_name', 'code95', 'urcode95', 'urcode96', 'avg_salary']]
    print("------ District Socio-economic Profile:")
    display(district_info)

    print("\n----------Client Loan Information:")
    display(filtered[['client_id', 'birth_number', 'status', 'amount', 'payments', 'date']].head(10))

# Step 6: Display
ui = widgets.VBox([district_dropdown, status_dropdown])
out = widgets.interactive_output(show_filtered_info, {'district': district_dropdown, 'status': status_dropdown})
display(ui, out)


Output()

In [153]:
df_New_Transaction["operation"].unique()

array(['CREDIT IN CASH', 'COLLECTION FROM ANOTHER BANK',
       'CASH WITHDRAWAL', 'REMITTANCE TO ANOTHER BANK',
       'CREDIT CARD WITHDRAWAL'], dtype=object)

In [174]:
#task-10
owners=df_New_Disposition[df_New_Disposition["type"]=="OWNER"]
owners_with_orders=pd.merge(owners,df_Order,on="account_id")
owners_with_loans=pd.merge(owners_with_orders,df_Loan,on="account_id")
account_merge=pd.merge(owners_with_loans,df_New_Account,on="account_id")
district_merge=pd.merge(account_merge,df_District,on="district_id")
data=district_merge.groupby("district_name").size().reset_index(name="count")
data

,district_name,count
0,Benesov,13
1,Beroun,9
2,Blansko,16
3,Breclav,21
4,Brno - mesto,47
...,...,...
72,Vsetin,13
73,Vyskov,18
74,Zdar nad Sazavou,17
75,Zlin,31


In [232]:
#task11
merge=pd.merge(df_New_Card,df_New_Disposition,on="disp_id")
merge_two=pd.merge(acc_gold_cards,df_New_Client,on="client_id")
merge_full=pd.merge(merge_two,df_District[["district_id","A3"]],on="district_id")
districts_Bohemia=['central Bohemia', 'south Bohemia', 'west Bohemia',
       'north Bohemia', 'east Bohemia']
districts_Moravia=['south Moravia',"north Moravia"]
data1=merge_full[(merge_full["A3"].isin(districts_Bohemia)) &(merge_full["type_x"]=="GOLD")&(merge_full["gender"]=="MALE")].groupby("A3").size().reset_index(name="count")
data2=merge_full[(merge_full["A3"].isin(districts_Moravia)) &(merge_full["type_x"]=="GOLD")&(merge_full["gender"]=="MALE")].groupby("A3").size().reset_index(name="count")
Bohemia_count=data1["count"].sum()
Moravia_count=data2["count"].sum()
print("count of Bohemia male customers who are having gold card--->",Bohemia_count)
print("count of Moravia male customers who are having gold card--->",Moravia_count)
if Bohemia_count >Moravia_count:
    print("Bohemia have more male customers possessing Gold cards in comparison of Moravia.")
else:
    print("Moravia have more male customers possessing Gold cards in comparison of Bohemia.")

count of Bohemia male customers who are having gold card---> 28
count of Moravia male customers who are having gold card---> 18
Bohemia have more male customers possessing Gold cards in comparison of Moravia.


In [243]:
#task-12
card_disp = pd.merge(df_New_Card, df_New_Disposition, on="disp_id")
card_loan = pd.merge(card_disp, df_Loan, on="account_id")
customers_with_card_and_loan = card_loan["client_id"].nunique()
print("Number of customers having credit card and availing loan facilities:", customers_with_card_and_loan)

Number of customers having credit card and availing loan facilities: 170


In [253]:
#task-13
merge1=pd.merge(df_New_Card,df_New_Disposition[["disp_id","account_id"]],on="disp_id")
merge2=pd.merge(merge1,df_Loan[["account_id","status"]],on="account_id")
cards=["CLASSIC","JUNIOR"]
filter_data=merge2[merge2["type"].str.upper().isin([c.upper() for c in cards])]
all_defaulted=filter_data["status"].eq("D").all()
print("YES" if all_defaulted else "FALSE")

FALSE


In [254]:
#task-14
owners = df_New_Disposition[df_New_Disposition["type"] == "OWNER"]
owners_clients = pd.merge(owners, df_New_Client[["client_id", "age_levels"]], on="client_id")
owners_loans = pd.merge(owners_clients, df_Loan[["account_id", "status"]], on="account_id")
loan_analysis = owners_loans.groupby(["age_levels", "status"]).size().reset_index(name="count")
loan_analysis


,age_levels,status,count
0,ADULT,A,51
1,ADULT,B,4
2,ADULT,C,109
3,ADULT,D,13
4,MIDDLE AGED,A,128
5,MIDDLE AGED,B,22
6,MIDDLE AGED,C,246
7,MIDDLE AGED,D,25
8,YOUTH,A,24
9,YOUTH,B,5
